In [237]:
import os
import cv2
import numpy as np

def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            # Resize and normalize
            img_resized = cv2.resize(img, (16, 16)) / 255.0
            # Flatten
            img_flattened = img_resized.flatten()
            images.append(img_flattened)
    return np.array(images)

# Load images for class 0 and 5
class_0_images = load_images_from_folder("../data/svm/train/0")
class_5_images = load_images_from_folder("../data/svm/train/1")

class_0_images.shape, class_5_images.shape

((2380, 768), (2380, 768))

In [238]:
X = np.concatenate((class_0_images, class_5_images), axis=0)
y = np.concatenate((-1 * np.ones(class_0_images.shape[0]), np.ones(class_5_images.shape[0])), axis=0)

In [239]:
def gaussian_kernel(x1, x2, gamma=0.001):
    return np.exp(-gamma * np.linalg.norm(x1 - x2) ** 2)

In [240]:
from scipy.spatial.distance import cdist

# Compute pairwise squared distances
squared_dists = cdist(X, X, 'sqeuclidean')

# Compute Gaussian Kernel values
gamma = 0.001
K = np.exp(-gamma * squared_dists)

# Compute the P matrix
P = np.outer(y, y) * K

In [241]:
from cvxopt import matrix, solvers
from numpy import double

q = -np.ones((X.shape[0], 1))
G = np.vstack([-np.eye(X.shape[0]), np.eye(X.shape[0])])
h = np.hstack([np.zeros(X.shape[0]), np.ones(X.shape[0]) * 1.0])  # Considering C = 1.0
A = y.reshape(1, -1)
b = np.zeros(1)

P = matrix(P)
q = matrix(q)
G = matrix(G)
h = matrix(h)
A = matrix(A)
b = matrix(b)

sol = solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -1.9393e+03 -1.4696e+04  9e+04  4e+00  3e-13
 1: -1.2731e+03 -1.0480e+04  2e+04  4e-01  4e-13
 2: -1.1845e+03 -2.6743e+03  1e+03  9e-13  4e-13
 3: -1.4357e+03 -2.1032e+03  7e+02  9e-13  3e-13
 4: -1.5025e+03 -1.9941e+03  5e+02  4e-13  3e-13
 5: -1.5733e+03 -1.8712e+03  3e+02  1e-13  3e-13
 6: -1.6192e+03 -1.7957e+03  2e+02  8e-13  3e-13
 7: -1.6452e+03 -1.7549e+03  1e+02  5e-13  3e-13
 8: -1.6661e+03 -1.7225e+03  6e+01  2e-13  3e-13
 9: -1.6769e+03 -1.7068e+03  3e+01  3e-13  3e-13
10: -1.6857e+03 -1.6947e+03  9e+00  1e-13  4e-13
11: -1.6890e+03 -1.6904e+03  1e+00  1e-12  4e-13
12: -1.6896e+03 -1.6897e+03  7e-02  2e-14  4e-13
13: -1.6896e+03 -1.6896e+03  2e-03  4e-12  4e-13
14: -1.6896e+03 -1.6896e+03  3e-05  5e-12  4e-13
Optimal solution found.


In [242]:
class_0_val_images = load_images_from_folder("../data/svm/val/0")
class_5_val_images = load_images_from_folder("../data/svm/val/1")

X_val = np.concatenate((class_0_val_images, class_5_val_images), axis=0)
y_val = np.concatenate((-1 * np.ones(class_0_val_images.shape[0]), np.ones(class_5_val_images.shape[0])), axis=0)

In [243]:
sv_indices = ((alphas > 1e-5) & (alphas < 1 - 1e-5)).flatten()
support_vectors = X[sv_indices]
support_vector_labels = y[sv_indices]
alphas_sv = alphas[sv_indices]

# Calculate kernel matrix for support vectors
K_sv = np.exp(-0.001 * cdist(support_vectors, support_vectors, 'sqeuclidean'))

# Calculate b for each support vector
b_values = support_vector_labels - np.sum(alphas_sv * support_vector_labels * K_sv, axis=1)

# Average over all b values
b = np.mean(b_values)

In [246]:
b = -7.728114

In [247]:
sv_indices = ((alphas > 1e-5)).flatten()
support_vectors = X[sv_indices]
support_vector_labels = y[sv_indices]
alphas_sv = alphas[sv_indices]

def predict_gaussian(X):
    # Calculate Gaussian Kernel values between each point in X and the support vectors
    K = np.exp(-0.001 * cdist(X, support_vectors, 'sqeuclidean'))
    
    # Calculate predictions using the kernel values, alphas, support vector labels, and b
    predictions = np.dot(K, alphas_sv.flatten() * support_vector_labels.flatten()) + b
    
    return np.sign(predictions).flatten()

y_val_pred = predict_gaussian(X_val)
np.mean(y_val_pred == y_val)

0.845